In [1]:
from pynq import Overlay
from pynq import PL

OL = Overlay("/home/xilinx/pynq/overlays/WholeNetwork/design_1_wrapper.bit")
dma = OL.axi_dma_0
network = OL.network_0

In [3]:
import numpy as np
import pynq

def get_pynq_buffer(shape, dtype):
    """ Simple function to call PYNQ's memory allocator with numpy attributes

    """
    return pynq.Xlnk().cma_array(shape, dtype)

matrix_in = get_pynq_buffer((1*28*28), np.int16)
matrix_out = get_pynq_buffer((1*28*28), np.int16)

matrix_in_addr = hex(matrix_in.physical_address)
matrix_out_addr = hex(matrix_out.physical_address)
print(matrix_in_addr, matrix_out_addr) 

0x18049000 0x1804a000


In [4]:
network.write(0x00, 1)
dma.sendchannel.transfer(matrix_in)
dma.recvchannel.transfer(matrix_out)
dma.sendchannel.wait()
dma.recvchannel.wait()

KeyboardInterrupt: 

## Address info  

    offset : bit   : function
    0x0000 : Control Signals
             bit 0 : ap_start (Read/Write/COH)  
             bit 1 : ap_done  (Read/COR)  
             bit 2 : ap_idle  (Read)  
             bit 7 : auto_restart (Read/Write)  
             others: reserved  

    0x0004 : Global Interrupt Enable Register
             bit 0 : Global Interrupt Enable(Read/Write)
             others: reserved

    0x0008 : IP Interrupt Enable Register(Read/Write)
             bit 0 : Channel 0 (ap_done)
             bit 1 : Channel 1 (ap_ready)
             others: reserved

    0x000c : IP Interrupt Status Register(Read/Write)
             bit 0 : Channel 0 (ap_done)
             bit 1 : Channel 1 (ap_ready)
             others: reserved

    0x0800 ~
    0x0fff : Memory "input_0_array" (784 * 16b)
           : Word n : bit[15: 0] - input_0_array[2n]
                    : bit[31:16] - input_0_array[2n+1]

    0x1000 ~
    0x17ff : Memory "output_0_array" (784 * 16b)
           : Word n : bit[15: 0] - input_0_array[2n]
                    : bit[31:16] - input_0_array[2n+1]

    (SC = Self Clear, COR = Clear on Read, TOW = Toggle on Write, COH = Clear on HandShake)


In [5]:
matrix_in.freebuffer()
matrix_out.freebuffer()